In [52]:
from ds_utils.imports import *

In [57]:
assert(keras.backend.backend()=='theano')

# Prework

Following this fast.ai [lesson](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/lesson4.ipynb) and [video](https://www.youtube.com/watch?v=V2h3IOBDvrA&feature=youtu.be&t=5761).

In [58]:
# get data
# !wget -O data/ml-latest-small.zip http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
# !unzip data/ml-latest-small.zip

# Preprocessing

In [59]:
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')

In [60]:
ratings.shape

(100004, 4)

In [61]:
ratings.sample(5)

,userId,movieId,rating,timestamp
44899,313,8636,3.5,1101034464
79278,547,3424,5.0,974780642
54740,391,1190,1.0,891533210
92601,615,1214,4.5,1408779685
61914,452,2121,3.0,976424511


In [62]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [63]:
userid2idx = {o: i for i, o in enumerate(users)}
movieid2idx = {o: i for i, o in enumerate(movies)}

In [64]:
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])

In [65]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()

In [66]:
n_factors = 50

In [67]:
np.random.seed = 42

In [68]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

# Dot product

In [22]:
user_in = keras.layers.Input(shape=(1, ), dtype='int64', name='user_in')

In [23]:
u = keras.layers.Embedding(
    input_dim=n_users,
    output_dim=n_factors,
    input_length=1,
    embeddings_regularizer=keras.regularizers.l2(l=1e-4))(user_in)

In [24]:
movie_in = keras.layers.Input(shape=(1, ), dtype='int64', name='movie_in')

In [25]:
m = keras.layers.Embedding(
    input_dim=n_movies,
    output_dim=n_factors,
    input_length=1,
    embeddings_regularizer=keras.regularizers.l2(l=1e-4))(movie_in)

In [26]:
x = keras.layers.merge([u, m], mode='dot')

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `sum`, `concatenate`, etc.
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/Keras-2.0.0-py3.6.egg/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [27]:
x = keras.layers.Flatten()(x)

In [28]:
model = keras.models.Model(inputs=[user_in, movie_in], outputs=x)

In [29]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='mse')

In [30]:
model.fit(x=[trn.userId, trn.movieId],
          y=trn.rating,
          batch_size=64,
          epochs=1,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80129 samples, validate on 19875 samples
Epoch 1/1
80129/80129 [==============================] - 7s - loss: 9.8370 - val_loss: 4.2825

In [31]:
model.optimizer.lr = 0.01

In [32]:
model.fit(
    x=[trn.userId, trn.movieId],
    y=trn.rating,
    batch_size=64,
    epochs=3,
    validation_data=([val.userId, val.movieId], val.rating), )

Train on 80129 samples, validate on 19875 samples
Epoch 1/3
80129/80129 [==============================] - 10s - loss: 3.1334 - val_loss: 2.8134

In [33]:
model.optimizer.lr = 0.001

In [34]:
model.fit(x=[trn.userId, trn.movieId],
          y=trn.rating,
          batch_size=64,
          epochs=6,
          validation_data=([val.userId, val.movieId], val.rating),
          shuffle=False)

Train on 80129 samples, validate on 19875 samples
Epoch 1/6
80129/80129 [==============================] - 9s - loss: 2.3389 - val_loss: 2.7524

# Bias

In [69]:
user_bias = keras.layers.Embedding(input_dim=n_users, output_dim=1, input_length=1)(user_in)
user_bias = keras.layers.Flatten()(user_bias)

In [37]:
movie_bias = keras.layers.Embedding(input_dim=n_movies, output_dim=1, input_length=1)(movie_in)
movie_bias = keras.layers.Flatten()(movie_bias)

In [38]:
x = keras.layers.merge([u, m], mode='dot')
x = keras.layers.Flatten()(x)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `sum`, `concatenate`, etc.
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/Keras-2.0.0-py3.6.egg/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [39]:
x = keras.layers.merge([x, user_bias], mode='sum')
x = keras.layers.merge([x, movie_bias], mode='sum')

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `sum`, `concatenate`, etc.
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/Keras-2.0.0-py3.6.egg/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `sum`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [79]:
type(x)

theano.tensor.var.TensorVariable

In [40]:
model = keras.models.Model(inputs=[user_in, movie_in], outputs=x)

In [41]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='mse')

In [42]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 50)         33550                                        
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1, 50)         453300                                       
___________________________________________________________________________________________

In [43]:
model.fit(x=[trn.userId, trn.movieId], y=trn.rating, batch_size=64, validation_data=([val.userId, val.movieId], val.rating))

Train on 80129 samples, validate on 19875 samples
Epoch 1/1
80129/80129 [==============================] - 10s - loss: 2.9555 - val_loss: 2.6054

In [44]:
model.optimizer.lr=0.01

In [45]:
model.fit(x=[trn.userId, trn.movieId], y=trn.rating, batch_size=64, epochs=10, validation_data=([val.userId, val.movieId], val.rating))

Train on 80129 samples, validate on 19875 samples
Epoch 1/10
80129/80129 [==============================] - 9s - loss: 2.1700 - val_loss: 2.2903

In [46]:
model.optimizer.lr=0.001

In [47]:
model.fit(x=[trn.userId, trn.movieId], y=trn.rating, batch_size=64, epochs=5, validation_data=([val.userId, val.movieId], val.rating))

Train on 80129 samples, validate on 19875 samples
Epoch 1/5
80129/80129 [==============================] - 10s - loss: 1.2493 - val_loss: 1.6083

# Inspect

In [92]:
type(user_in)

theano.tensor.var.TensorVariable

In [93]:
user_in_layer = model.get_layer(name='user_in')

In [94]:
user_in_layer.input_shape

(None, 1)

In [95]:
user_in_layer.output_shape

(None, 1)

In [96]:
model.get_layer(index=2).output_shape

(None, 1, 50)

# NN

In [133]:
x = keras.layers.merge([u, m], mode='concat')

x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(70, activation='relu')(x)
x = keras.layers.Dropout(0.75)(x)
x = keras.layers.Dense(1)(x)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `sum`, `concatenate`, etc.
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/Keras-2.0.0-py3.6.egg/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [138]:
nn = keras.models.Model(inputs=[user_in, movie_in], outputs=x)

In [140]:
nn.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='mse')

In [ ]:
nn.fit(x=[trn.userId, trn.movieId],
          y=trn.rating,
          batch_size=64,
          epochs=10,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80148 samples, validate on 19856 samples
Epoch 1/10
80148/80148 [==============================] - 36s - loss: 2.5843 - val_loss: 1.2265

# Get Parts of Model

## Get bias

In [157]:
mdl_movie_bias = keras.models.Model(inputs=movie_in, outputs=movie_bias)

In [158]:
mdl_movie_bias.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
movie_in (InputLayer)        (None, 1)                 0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 1, 1)              9066      
_________________________________________________________________
flatten_3 (Flatten)          (None, 1)                 0         
Total params: 9,066.0
Trainable params: 9,066.0
Non-trainable params: 0.0
_________________________________________________________________


In [159]:
movies

array([  31, 1029, 1061, ...,  129, 4736, 6425])

In [160]:
mdl_movie_bias.predict(np.random.choice(ratings.movieId, 5))

array([[ 0.47997302],
       [ 0.53042865],
       [ 0.36906186],
       [ 0.47014344],
       [ 0.34738836]], dtype=float32)

## Get embedding

In [161]:
mdl_movie_embedding = keras.models.Model(inputs=movie_in, outputs=m)

In [162]:
mdl_movie_embedding.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
movie_in (InputLayer)        (None, 1)                 0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1, 50)             453300    
Total params: 453,300.0
Trainable params: 453,300.0
Non-trainable params: 0.0
_________________________________________________________________


In [165]:
mdl_movie_embedding.predict(np.random.choice(ratings.movieId, 2))

array([[[-0.00179446, -0.00615287, -0.00737933, -0.00525931,  0.05306772,
         -0.02318139,  0.01056315, -0.00142497, -0.00665551,  0.04978988,
         -0.09068959,  0.03001247, -0.00682778, -0.00604128,  0.00096218,
          0.01051673, -0.00463524,  0.03267124,  0.02768091,  0.02939803,
         -0.0375102 , -0.02189777,  0.00177513, -0.01578556,  0.0043972 ,
          0.02179634, -0.05279788,  0.00686759, -0.05015779,  0.06274927,
         -0.01546975,  0.00392498, -0.01336711,  0.00853537, -0.05243705,
          0.04487561,  0.0318293 , -0.01649065,  0.02221778, -0.01103533,
          0.02412142, -0.0005086 ,  0.02953145,  0.01781693,  0.01305428,
         -0.01569401, -0.0186593 , -0.03651908, -0.0233814 , -0.01668829]],

       [[-0.00780246,  0.00482874,  0.02858766,  0.05842306, -0.02153516,
         -0.02073379,  0.06724703,  0.0138449 ,  0.02037653,  0.00073729,
          0.09508857, -0.01011115,  0.0290802 ,  0.02146183,  0.00874694,
         -0.0236809 ,  0.01306115, 